In [1]:
import numpy as np
from scipy.optimize import linprog

from mis_utils import *

In [2]:
# Definiciones de constantes

node_names = np.array(('s', '2', '3', '4', '5', 't'))

# Balances: Sale un producto desde s y llega hasta t
beq = np.array([1, 0, 0, 0, 0, -1])

NN = np.array([[0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 0]])

# Matrices resultantes de NN a NA
Aeq, arc_idxs = nn2na(NN, node_names = node_names, show_results = True)

# Guardo los nombres de los nodo-arco-nodo posibles
nan_names = get_col_names(NN, node_names, as_numpy=True)

# Restricciones, l <= x <= u
# Entre 0 y 1 sería lo teóricamente correcto, aunque como hay un límite que como el modelo indica, es de un producto
min_bound = 0
max_bound = 1
bounds = tuple([(min_bound, max_bound) for arcs in range(0, Aeq.shape[1])])

[[0 1]
 [0 2]
 [1 3]
 [1 5]
 [2 4]
 [3 5]
 [4 5]]
Input: 
 [[0 1 1 0 0 0]
 [0 0 0 1 0 1]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]]

Column names: s2-s3-24-2t-35-4t-5t

Output: 
 [[ 1  1  0  0  0  0  0]
 [-1  0  1  1  0  0  0]
 [ 0 -1  0  0  1  0  0]
 [ 0  0 -1  0  0  1  0]
 [ 0  0  0  0 -1  0  1]
 [ 0  0  0 -1  0 -1 -1]]



In [3]:
# Vector de costos
# Con el orden de los nombres de columnas, sacado de los resultados. Column names: s2-s3-24-2t-35-4t-5t
C = np.array([2, 2, 2, 5, 2, 1, 2])

In [4]:
# Resumen
print('## Optimizer inputs ## \n\n'
      'Cost vector: %s \n\n'
      'Columns: %s \n\n'
      'A_eq Node-Arc matrix:\n%s \n\n'
      'b_eq demand-supply vector: %s \n\n'
      'Bounds of each X arc variable: %s \n' % (C, nan_names, Aeq, beq, bounds))

## Optimizer inputs ## 

Cost vector: [2 2 2 5 2 1 2] 

Columns: ['s2' 's3' '24' '2t' '35' '4t' '5t'] 

A_eq Node-Arc matrix:
[[ 1  1  0  0  0  0  0]
 [-1  0  1  1  0  0  0]
 [ 0 -1  0  0  1  0  0]
 [ 0  0 -1  0  0  1  0]
 [ 0  0  0  0 -1  0  1]
 [ 0  0  0 -1  0 -1 -1]] 

b_eq demand-supply vector: [ 1  0  0  0  0 -1] 

Bounds of each X arc variable: ((0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)) 



In [5]:
# Optimización
res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds)

/home/diego/.virtualenvs/OS11/lib/python3.6/site-packages/ipykernel_launcher.py:2: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  
/home/diego/.virtualenvs/OS11/lib/python3.6/site-packages/ipykernel_launcher.py:2: OptimizeWarning: Solving system with option 'cholesky':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'cholesky' to False.
  
/home/diego/.virtualenvs/OS11/lib/python3.6/site-packages/ipykernel_launcher.py:2: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
  
/home/diego/.virtualenvs/OS11/lib/python3.6/site-packages/ipykernel_launcher.py:2: OptimizeWarning: Solving system w

In [6]:
res.x.astype(int)

array([1, 0, 1, 0, 0, 1, 0])

In [7]:
get_col_names(NN, node_names, as_numpy=True)

array(['s2', 's3', '24', '2t', '35', '4t', '5t'], dtype='<U32')

In [8]:
nan_names

array(['s2', 's3', '24', '2t', '35', '4t', '5t'], dtype='<U32')

In [9]:
arc_idxs

[(0, 1), (0, 2), (1, 3), (1, 5), (2, 4), (3, 5), (4, 5)]

In [10]:
selarcs = get_selected_arcs(res.x, nan_names)

In [11]:
nan_names[2]

'24'

In [12]:
# GET THE SOLUTION:

print('## Results ##')
print('The raw solution will be: %s' % res.x)
print('The arcs that make the shortest path will be (from, to): %s' % selarcs)
print('The minimum cost will be: %0.2f ' % res.fun)

## Results ##
The raw solution will be: [1.00000000e+00 1.48688336e-13 1.00000000e+00 8.05223839e-14
 1.48676829e-13 1.00000000e+00 1.48686035e-13]
The arcs that make the shortest path will be (from, to): ['s2', '24', '4t']
The minimum cost will be: 5.00 
